In [71]:
# libraries
import math
import numpy as np

In [32]:
def read_file_info(filename):
    
    tsp_info = {
        'name' : '',
        'comment' : '',
        'dimension' : '',
        'original_structure' : '',
        'original_substructure' : '',
    }


    with open(filename, 'r') as file:
        lines = file.readlines()

        for line in lines:
            line = line.strip()
            line = line.replace(':', ' ')

            if line.startswith('EDGE_WEIGHT_SECTION'):
                # Explicit
                break
            elif line.startswith('NODE_COORD_SECTION'):
                # EUC
                break
            else:
                if line.startswith('NAME'):
                    tsp_info['name'] = line.split()[1]
                    
                elif line.startswith('COMMENT'):
                    tsp_info['comment'] = ' '.join(line.split()[1:])
                
                elif line.startswith('DIMENSION'):
                    tsp_info['dimension'] = int(line.split()[1])

                elif line.startswith('EDGE_WEIGHT_TYPE'):
                    tsp_info['original_structure'] = line.split()[1]
                
                elif line.startswith('EDGE_WEIGHT_FORMAT'):
                    tsp_info['original_substructure'] = line.split()[1]
    
    return tsp_info 

In [ ]:
def calculate_distance_euc_2d(city1, city2, decimals=0):
    distance = np.sqrt((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)
    return math.ceil(distance)

In [34]:
def read_file_euc_2d(filename):

    cities = []

    with open(filename, 'r') as f:
        lines = f.readlines()
        read_cities = False

        for line in lines: 
            if line.startswith("EOF"):
                break
            elif read_cities:
                city_parts = line.split()
                cities.append((int(city_parts[0]), float(city_parts[1]), float(city_parts[2])))
            else:
                if line.startswith("NODE_COORD_SECTION"):
                    read_cities = True


    num_cities = len(cities)
    distances = np.zeros((num_cities, num_cities), dtype=int)

    for i in range(num_cities):
        for j in range(num_cities):
            tmp_value = calculate_distance_euc_2d(cities[i], cities[j])
            distances[i][j] = int(tmp_value)

    return distances

In [ ]:
def read_file_explicit_dummy(filename, tsp_info):
    
    lower_diag = []

    with open(filename, 'r') as f:
        lines = f.readlines()
        read_diag = False

        for line in lines: 
            if line.startswith("EOF"):
                break
            elif read_diag:
                diag_parts = line.split()
                lower_diag.extend(diag_parts)
            else:
                if line.startswith("EDGE_WEIGHT_SECTION"):
                    read_diag = True

    distances = np.zeros((tsp_info['dimension'], tsp_info['dimension']), dtype=int)
    index = 0
    for i in range(tsp_info['dimension'] - 2):
        for j in range(i + 1):
            distances[i, j] = lower_diag[index]
            distances[j, i] = lower_diag[index]
            index += 1

    return distances

In [35]:
def read_file_explicit_upper_row(filename, tsp_info):
    
    upper_diag = []

    with open(filename, 'r') as f:
        lines = f.readlines()
        read_diag = False

        for line in lines: 
            if line.startswith("EOF"):
                break
            elif read_diag:
                diag_parts = line.split()
                upper_diag.extend(diag_parts)
            else:
                if line.startswith("EDGE_WEIGHT_SECTION"):
                    read_diag = True

    read_diag_index = 0
    distances = np.zeros((tsp_info['dimension'], tsp_info['dimension']), dtype=int)

    for i in range(0, tsp_info['dimension']):
        for  j in range(i, tsp_info['dimension']):
            if (i == j):
                distances[i][j] = 0
            else:
                distances[i][j] = upper_diag[read_diag_index]
                distances[j][i] = upper_diag[read_diag_index]
                read_diag_index = read_diag_index + 1
 

    return distances


In [47]:
def read_file_explicit_lower_dialog_row(filename, tsp_info):
    
    lower_diag = []

    with open(filename, 'r') as f:
        lines = f.readlines()
        read_diag = False

        for line in lines: 
            if line.startswith("EOF"):
                break
            elif read_diag:
                diag_parts = line.split()
                lower_diag.extend(diag_parts)
            else:
                if line.startswith("EDGE_WEIGHT_SECTION"):
                    read_diag = True

    read_diag_index = 0
    distances = np.zeros((tsp_info['dimension'], tsp_info['dimension']), dtype=int)

    for j in range(0, tsp_info['dimension']):
        for  i in range(0, j + 1): 
            if (i == j):
                distances[i][j] = 0
                read_diag_index = read_diag_index + 1
            else: 
                distances[i][j] = lower_diag[read_diag_index]
                distances[j][i] = lower_diag[read_diag_index]
                read_diag_index = read_diag_index + 1
 

    return distances


In [ ]:
def dummy(filename, tsp_info):

    return False

In [37]:
def write_distance_matrix(tsp_info, distances, output_filename):

    with open(output_filename, 'w') as f:
        f.write(f"NAME : {tsp_info['name']}\n")
        f.write(f"TYPE : TSP\n")
        f.write(f"COMMENT : {tsp_info['comment']}\n")
        f.write(f"DIMENSION : {tsp_info['dimension']}\n")
        f.write(f"EDGE_WEIGHT_TYPE : EXPLICIT\n")
        f.write(f"EDGE_WEIGHT_FORMAT : FULL_MATRIX\n")
        f.write(f"EDGE_WEIGHT_SECTION\n")

        for i in range(len(distances)):
            for j in range(len(distances[i])):
                f.write(f"{distances[i][j]} ")
            f.write("\n")

        f.write("EOF\n")

In [38]:
def read_file_content(filename, tsp_info, output_filename):
    
    if tsp_info['original_structure'] == 'EUC_2D':
        file_distances = read_file_euc_2d(filename)
    elif tsp_info['original_structure'] == 'EXPLICIT':
        # if tsp_info['original_substructure'] == 'FULL_MATRIX':
        #     # nothing
        #     file_dist = read_file_explicit_full_matrix(filename)
        if tsp_info['original_substructure'] == 'UPPER_ROW':
            file_distances = read_file_explicit_upper_row(filename, tsp_info)
        elif tsp_info['original_substructure'] == 'LOWER_DIAG_ROW':
            file_distances = read_file_explicit_lower_dialog_row(filename, tsp_info)
        else:
            return False
    
    else:
        return False

    write_distance_matrix(tsp_info, file_distances, output_filename)


In [70]:
# TSP Problems selected for Analysis:
# 
# SMALL:
# 
# gr17.tsp, 17, 17-city problem (Groetschel)
# bayg29.tsp, 29, 29 Cities in Bavaria, geographical distances (Groetschel,Juenger,Reinelt)
# swiss42.tsp, 42, 42 Staedte Schweiz (Fricker)
# dantzig42.tsp, 42, 42 cities (Dantzig)
# gr48.tsp, 48, 48-city problem (Groetschel)
# 
# 
# MEDIUM:
# 
# brazil58, 58, 58 cities in Brazil (Ferreira),
# st70.tsp, 70, 70-city problem (Smith/Thompson)
# eil76.tsp, 76, 76-city problem (Christofides/Eilon)
# rat99.tsp, 99, Rattled grid (Pulleyblank)
# kroA.tsp, 100, 100-city problem A (Krolak/Felts/Nelson)
# 
# 
# LARGE: 
# 
# lin105.tsp, 105, 105-city problem (Subproblem of lin318)
# pr107.tsp, 107,  107-city problem (Padberg/Rinaldi)
# bier127.tsp, 127, 127 Biergaerten in Augsburg (Juenger/Reinelt)
# pr136.tsp, 136, 136-city problem (Padberg/Rinaldi)
# kroB150.tsp, 150, 150-city problem B (Krolak/Felts/Nelson)

In [53]:
filename = "tsplib_instances_project/01_small/gr17.tsp"
filename_new = "tsplib_converted_instances/01_small/gr17_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [54]:
filename = "tsplib_instances_project/01_small/bayg29.tsp"
filename_new = "tsplib_converted_instances/01_small/bayg29_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [56]:
filename = "tsplib_instances_project/01_small/dantzig42.tsp"
filename_new = "tsplib_converted_instances/01_small/dantzig42_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [57]:
filename = "tsplib_instances_project/01_small/gr48.tsp"
filename_new = "tsplib_converted_instances/01_small/gr48_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [58]:
filename = "tsplib_instances_project/02_medium/brazil58.tsp"
filename_new = "tsplib_converted_instances/02_medium/brazil58_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [59]:
filename = "tsplib_instances_project/02_medium/st70.tsp"
filename_new = "tsplib_converted_instances/02_medium/st70_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [60]:
filename = "tsplib_instances_project/02_medium/eil76.tsp"
filename_new = "tsplib_converted_instances/02_medium/eil76_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [61]:
filename = "tsplib_instances_project/02_medium/rat99.tsp"
filename_new = "tsplib_converted_instances/02_medium/rat99_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [62]:
filename = "tsplib_instances_project/02_medium/kroA100.tsp"
filename_new = "tsplib_converted_instances/02_medium/kroA100_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [64]:
filename = "tsplib_instances_project/03_large/lin105.tsp"
filename_new = "tsplib_converted_instances/03_large/lin105_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [65]:
filename = "tsplib_instances_project/03_large/pr107.tsp"
filename_new = "tsplib_converted_instances/03_large/pr107_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [66]:
filename = "tsplib_instances_project/03_large/bier127.tsp"
filename_new = "tsplib_converted_instances/03_large/bier127_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [67]:
filename = "tsplib_instances_project/03_large/pr136.tsp"
filename_new = "tsplib_converted_instances/03_large/pr136_converted.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)

In [68]:
filename = "tsplib_instances_project/03_large/kroB150.tsp"
filename_new = "tsplib_converted_instances/03_large/kroB150.tsp" 
tsp_info = read_file_info(filename)
read_file_content(filename, tsp_info, filename_new)